### Preparando o ambiente

In [ ]:
import geopandas as gpd
import pandas as pd
import h3
from IPython.display import clear_output
import unicodedata

# Carregando o arquivo de malhas
brasil = gpd.read_file("data/brasil.shp")

In [ ]:
def aggregate_values_proportional_efficient(
    gdf: gpd.GeoDataFrame, 
    gdf_code: str, 
    df: pd.DataFrame, 
    df_code: str, 
    columns: list[str]
) -> gpd.GeoDataFrame:
    """
    Aggregate values from a DataFrame to a GeoDataFrame proportionally to the area overlap,
    using a more efficient, vectorized approach.
    
    Parameters:
        gdf: GeoDataFrame to which values will be aggregated.
        gdf_code: Column name in gdf containing unique identifiers.
        df: DataFrame containing values to be aggregated.
        df_code: Column name in df containing identifiers to match with gdf.
        columns: List of column names in df to aggregate.
    
    Returns:
        A new GeoDataFrame with the aggregated values.
    """
    gdf_agg = gdf[[gdf_code, 'geometry']].copy()
    
    gdf_agg['area'] = gdf_agg.geometry.area
    
    gdf_agg['total_area_for_code'] = gdf_agg.groupby(gdf_code)['area'].transform('sum')
    
    gdf_agg['proportion'] = gdf_agg['area'].div(gdf_agg['total_area_for_code']).fillna(0)
    
    df_sum = df.groupby(df_code)[columns].sum()
    
    result = gdf_agg.merge(df_sum, left_on=gdf_code, right_index=True, how='left')
    
    result[columns] = result[columns].multiply(result['proportion'], axis=0)

    final_gdf = gdf.copy()
    final_gdf[columns] = result.groupby(final_gdf.index)[columns].sum().fillna(0)

    return final_gdf

# Agregando os dados do IBGE

In [6]:
df_ibge = pd.read_csv("data/Populacao_residente_por_bairro3.csv")

agg_columns =[
    "População Total", "População 0 a 4 anos", 
    "População Menos de 1 ano", "População 1 a 4 anos", "População 0 a 5 anos", 
    "População 10 a 14 anos", "População 15 a 19 anos", "População 15 a 17 anos", 
    "População 18 a 19 anos", "População 20 a 24 anos", "População 25 a 29 anos", 
    "População 30 a 39 anos", "População 40 a 49 anos", "População 50 a 59 anos", 
    "População 60 a 69 anos", "População 70 anos ou mais", "População Masculina Total", 
    "População Masculina 0 a 4 anos", "População Masculina Menos de 1 ano", 
    "População Masculina 1 a 4 anos", "População Masculina 0 a 5 anos", 
    "População Masculina 10 a 14 anos", "População Masculina 15 a 19 anos", 
    "População Masculina 15 a 17 anos", "População Masculina 18 a 19 anos", 
    "População Masculina 20 a 24 anos", "População Masculina 25 a 29 anos", 
    "População Masculina 30 a 39 anos", "População Masculina 40 a 49 anos", 
    "População Masculina 50 a 59 anos", "População Masculina 60 a 69 anos", 
    "População Masculina 70 anos ou mais", "População Feminina Total", 
    "População Feminina 0 a 4 anos", "População Feminina Menos de 1 ano", 
    "População Feminina 1 a 4 anos", "População Feminina 0 a 5 anos", 
    "População Feminina 10 a 14 anos", "População Feminina 15 a 19 anos", 
    "População Feminina 15 a 17 anos", "População Feminina 18 a 19 anos", 
    "População Feminina 20 a 24 anos", "População Feminina 25 a 29 anos", 
    "População Feminina 30 a 39 anos", "População Feminina 40 a 49 anos", 
    "População Feminina 50 a 59 anos", "População Feminina 60 a 69 anos", 
    "População Feminina 70 anos ou mais", "População Urbana Total", 
    "População Urbana 0 a 4 anos", "População Urbana Menos de 1 ano", 
    "População Urbana 1 a 4 anos", "População Urbana 0 a 5 anos", 
    "População Urbana 10 a 14 anos", "População Urbana 15 a 19 anos", 
    "População Urbana 15 a 17 anos", "População Urbana 18 a 19 anos", 
    "População Urbana 20 a 24 anos", "População Urbana 25 a 29 anos", 
    "População Urbana 30 a 39 anos", "População Urbana 40 a 49 anos", 
    "População Urbana 50 a 59 anos", "População Urbana 60 a 69 anos", 
    "População Urbana 70 anos ou mais", "População Urbana Masculina Total", 
    "População Urbana Masculina 0 a 4 anos", "População Urbana Masculina Menos de 1 ano", 
    "População Urbana Masculina 1 a 4 anos", "População Urbana Masculina 0 a 5 anos", 
    "População Urbana Masculina 10 a 14 anos", "População Urbana Masculina 15 a 19 anos", 
    "População Urbana Masculina 15 a 17 anos", "População Urbana Masculina 18 a 19 anos", 
    "População Urbana Masculina 20 a 24 anos", "População Urbana Masculina 25 a 29 anos", 
    "População Urbana Masculina 30 a 39 anos", "População Urbana Masculina 40 a 49 anos", 
    "População Urbana Masculina 50 a 59 anos", "População Urbana Masculina 60 a 69 anos", 
    "População Urbana Masculina 70 anos ou mais", "População Urbana Feminina Total", 
    "População Urbana Feminina 0 a 4 anos", "População Urbana Feminina Menos de 1 ano", 
    "População Urbana Feminina 1 a 4 anos", "População Urbana Feminina 0 a 5 anos", 
    "População Urbana Feminina 10 a 14 anos", "População Urbana Feminina 15 a 19 anos", 
    "População Urbana Feminina 15 a 17 anos", "População Urbana Feminina 18 a 19 anos", 
    "População Urbana Feminina 20 a 24 anos", "População Urbana Feminina 25 a 29 anos", 
    "População Urbana Feminina 30 a 39 anos", "População Urbana Feminina 40 a 49 anos", 
    "População Urbana Feminina 50 a 59 anos", "População Urbana Feminina 60 a 69 anos", 
    "População Urbana Feminina 70 anos ou mais", "População Rural Total", 
    "População Rural 0 a 4 anos", "População Rural Menos de 1 ano", 
    "População Rural 1 a 4 anos", "População Rural 0 a 5 anos", 
    "População Rural 10 a 14 anos", "População Rural 15 a 19 anos", 
    "População Rural 15 a 17 anos", "População Rural 18 a 19 anos", 
    "População Rural 20 a 24 anos", "População Rural 25 a 29 anos", 
    "População Rural 30 a 39 anos", "População Rural 40 a 49 anos", 
    "População Rural 50 a 59 anos", "População Rural 60 a 69 anos", 
    "População Rural 70 anos ou mais", "População Rural Masculina Total", 
    "População Rural Masculina 0 a 4 anos", "População Rural Masculina Menos de 1 ano", 
    "População Rural Masculina 1 a 4 anos", "População Rural Masculina 0 a 5 anos", 
    "População Rural Masculina 10 a 14 anos", "População Rural Masculina 15 a 19 anos", 
    "População Rural Masculina 15 a 17 anos", "População Rural Masculina 18 a 19 anos", 
    "População Rural Masculina 20 a 24 anos", "População Rural Masculina 25 a 29 anos", 
    "População Rural Masculina 30 a 39 anos", "População Rural Masculina 40 a 49 anos", 
    "População Rural Masculina 50 a 59 anos", "População Rural Masculina 60 a 69 anos", 
    "População Rural Masculina 70 anos ou mais", "População Rural Feminina Total", 
    "População Rural Feminina 0 a 4 anos", "População Rural Feminina Menos de 1 ano", 
    "População Rural Feminina 1 a 4 anos", "População Rural Feminina 0 a 5 anos", 
    "População Rural Feminina 10 a 14 anos", "População Rural Feminina 15 a 19 anos", 
    "População Rural Feminina 15 a 17 anos", "População Rural Feminina 18 a 19 anos", 
    "População Rural Feminina 20 a 24 anos", "População Rural Feminina 25 a 29 anos", 
    "População Rural Feminina 30 a 39 anos", "População Rural Feminina 40 a 49 anos", 
    "População Rural Feminina 50 a 59 anos", "População Rural Feminina 60 a 69 anos", 
    "População Rural Feminina 70 anos ou mais"
]

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7952\2390594438.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ibge = pd.read_csv("data/Populacao_residente_por_bairro3.csv")


In [7]:
clear_output()
gdf_agg = aggregate_values_proportional(brasil, 'CD_BAIRRO', df_ibge, 'Código do Bairro', agg_columns)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7952\4139919205.py:19: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = result.geometry.area
c:\Users\Administrator\Documents\projeto-paradigmas-ml\.venv\Lib\site-packages\geopandas\geodataframe.py:1968: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\Administrator\Documents\projeto-paradigmas-ml\.venv\Lib\site-packages\geopandas\geodataframe.py:1968: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at o

No match found for code: Total


In [ ]:
gdf_agg.to_parquet("data/brasil_agg.parquet", index=False)

In [ ]:
ufs = gdf_agg['CD_UF'].unique()

for uf in ufs:
    uf_data = gdf_agg[gdf_agg['CD_UF'] == uf].copy()
    uf_name = uf_data['NM_UF'].iloc[0]

    def remove_accents(text):
        return ''.join(c for c in unicodedata.normalize('NFD', text)
                      if unicodedata.category(c) != 'Mn')

    uf_name_clean = remove_accents(uf_name).replace(' ', '_')
    uf_data.to_parquet(f"data/uf_{uf}_{uf_name_clean}.parquet", index=False)
    
    print(f"Dataset criado para {uf_name} (UF {uf}) com {len(uf_data)} registros")

Dataset criado para Acre (UF 12) com 2215 registros
Dataset criado para Alagoas (UF 27) com 6359 registros
Dataset criado para Amazonas (UF 13) com 10729 registros
Dataset criado para Amapá (UF 16) com 1492 registros
Dataset criado para Bahia (UF 29) com 30985 registros
Dataset criado para Ceará (UF 23) com 20979 registros
Dataset criado para Distrito Federal (UF 53) com 5418 registros
Dataset criado para Espírito Santo (UF 32) com 8706 registros
Dataset criado para Goiás (UF 52) com 12861 registros
Dataset criado para Maranhão (UF 21) com 16301 registros
Dataset criado para Minas Gerais (UF 31) com 51387 registros
Dataset criado para Mato Grosso do Sul (UF 50) com 6117 registros
Dataset criado para Mato Grosso (UF 51) com 9381 registros
Dataset criado para Pará (UF 15) com 16714 registros
Dataset criado para Paraíba (UF 25) com 9639 registros
Dataset criado para Pernambuco (UF 26) com 19578 registros
Dataset criado para Piauí (UF 22) com 7340 registros
Dataset criado para Paraná (UF 4